# Classification on `emnist`

## 1. Create `Readme.md` to document your work

Explain your choices, process, and outcomes.

## 2. Classify ~~all symbols~~ letters a -> g

### Subset the data

Select only the lowercase letters (a, b, ..., g) for classification

### Choose a model

Your choice of model! Choose wisely...

### Train away!

Is do you need to tune any parameters? Is the model expecting data in a different format?

### Evaluate the model

Evaluate the models on the test set, analyze the confusion matrix to see where the model performs well and where it struggles.

### Investigate subsets

On which classes does the model perform well? Poorly? Evaluate again, excluding easily confused symbols (such as 'O' and '0').

### Improve performance

Brainstorm for improving the performance. This could include trying different architectures, adding more layers, changing the loss function, or using data augmentation techniques.

## 3. Model showdown: upper vs lowercase on abcXYZ

### Subset the data

Select out the set of upper- and lowercase (a, b, c, x, y z, A, B, C, X, Y, Z). Note that some of these classes can be confusing (e.g., x and y).

### Train and tune models

Perform a full model training and hyperparameter tuning.

1. Select candidate models, hyperparameter options, and evaluation metric
2. Set aside a validation hold-out dataset
3. Train models over K splits (use k-fold or train/test split)
    1. Split train using k-fold with the number of folds equal to the number of parameter combinations
    2. Train on k-fold split
    3. Record performance of each set of parameters
    4. Use winning set of parameters to train model on full training set
    5. Record each model's performance on that split's test set
4. Evaluate model performance and promote one model as the winner
5. Train winning model on both train + test
6. Check model performance on the validation hold-out


## 4. (_Optional_) Model comparison: classify even vs odd

**NOTE:** This is a larger dataset (~400k rows) so it will require more memory and time to train models on it. 

Alternatively, you can train models on smaller subsets of the data to get a feel for which models perform better than others. Then train the winning model on the full dataset and validate against the hold-out.

### Subset the data

Select only digits and add a column for 'is_even'. Be sure to create a validation hold-out dataset for later.

### Build and compare models

Train at least two different models, compare the results and choose a winner based on an evaluation metric of your choice.

In [ ]:
%pip install -q emnist pandas pyarrow numpy matplotlib seaborn scikit-learn xgboost tensorflow
%reset -f

In [ ]:
display("HI!")

In [1]:
# Import packages
import os
import string
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import emnist
from IPython.display import display, Markdown

# ML packages
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# XGBoost (SVM)
from xgboost import XGBClassifier
# Deep Learning
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Flatten

# Constants
SIZE = 28

In [2]:
# Define helper functions
def int_to_char(label):
    """Convert an integer label to the corresponding uppercase character."""
    if label < 10:
        return str(label)
    elif label < 36:
        return chr(label - 10 + ord('A'))
    else:
        return chr(label - 36 + ord('a'))

def show_image(row):
    """Display a single image and its corresponding label."""
    image = row['image']
    label = row['label']
    plt.imshow(image, cmap='gray')
    plt.title('Label: ' + int_to_char(label))
    plt.axis('off')
    plt.show()

def show_grid(data, title=None, num_cols=5, figsize=(20, 10)):
    """
    Display a list of images as a grid of num_cols columns.
    images: a list of images, each represented as a 28x28 numpy array
    labels: a list of labels, one for each image
    title: (optional) a title for the plot
    num_cols: (optional) number of columns to use in the grid
    figsize: (optional) size of the figure
    """
    num_images = len(data)
    num_rows = (num_images - 1) // num_cols + 1
    fig, axes = plt.subplots(num_rows, num_cols, figsize=figsize)
    if title is not None:
        fig.suptitle(title, fontsize=16)
    for i in range(num_rows):
        for j in range(num_cols):
            index = i * num_cols + j
            if index < num_images:
                axes[i, j].imshow(data.iloc[index]['image'], cmap='gray')
                axes[i, j].axis('off')
                label = int_to_char(data.iloc[index]['label'])
                axes[i, j].set_title(label)
    plt.show()

# Get a random image of a given label from the dataset
def get_image_by_label(data, label):
    """Get a random image of a given label from the dataset."""
    images = data[data['label'] == label]['image'].tolist()
    return random.choice(images)

# Plot the training and validation accuracy during the training of a model
def plot_accuracy(history):
    """Plot the training and validation accuracy during the training of a model."""
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, 'bo', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Plot the training and validation loss during the training of a model
def plot_loss(history):
    """Plot the training and validation loss during the training of a model."""
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# Normalize the pixel values of the images in the dataset to have zero mean and unit variance
# This is a common preprocessing step for neural networks, but may not be necessary in all cases
def normalize_images(images):
    """Normalize the pixel values of the images in the dataset to have zero mean and unit variance."""
    images = np.array(images)
    mean = images.mean()
    std = images.std()
    images = (images - mean) / std
    return images.tolist()

# Display metrics for a model
def display_metrics(task, model_name, metrics_dict):
    """Display performance metrics and confusion matrix for a model."""
    metrics_df = pd.DataFrame()
    cm_df = pd.DataFrame()
    for key, value in metrics_dict[task][model_name].items():
        if type(value) == np.ndarray:
            cm_df = pd.DataFrame(value, index=['actual a', 'actual b','actual c','actual d','actual e','actual f','actual g'], columns=['predicted a', 'predicted b','predicted c','predicted d','predicted e','predicted f','predicted g'])
        else:
            metrics_df[key] = [value]
    display(Markdown(f'# Performance Metrics: {model_name}'))
    display(metrics_df)
    display(Markdown(f'# Confusion Matrix: {model_name}'))
    display(cm_df)

def display_metrics2(task, model_name, metrics_dict):
    """Display performance metrics and confusion matrix for a model."""
    metrics_df = pd.DataFrame()
    cm_df = pd.DataFrame()
    for key, value in metrics_dict[task][model_name].items():
        if type(value) == np.ndarray:
            cm_df = pd.DataFrame(value, index=['actual a', 'actual b','actual c','actual A','actual B','actual C','actual x','actual y','actual z','actual X','actual Y','actual Z'], columns=['predicted a', 'predicted b','predicted c','predicted A','predicted B','predicted C','predicted x','predicted y','predicted z','predicted X','predicted Y','predicted Z'])
        else:
            metrics_df[key] = [value]
    display(Markdown(f'# Performance Metrics: {model_name}'))
    display(metrics_df)
    display(Markdown(f'# Confusion Matrix: {model_name}'))
    display(cm_df)

In [3]:
# Load data

# Extract the training split as images and labels
image, label = emnist.extract_training_samples('byclass')

# Add columns for each pixel value (28x28 = 784 columns)
emnist_train = pd.DataFrame()

# Add a column with the image data as a 28x28 array
emnist_train['image'] = list(image)
emnist_train['image_flat'] = emnist_train['image'].apply(lambda x: np.array(x).reshape(-1))

# Add a column showing the label
emnist_train['label'] = label

# Convert labels to characters
class_label = np.array([int_to_char(l) for l in label])

# Add a column with the character corresponding to the label
emnist_train['class'] = class_label

# Repeat for the test split
image, label = emnist.extract_test_samples('byclass')
class_label = np.array([int_to_char(l) for l in label])
emnist_test = pd.DataFrame()
emnist_test['image'] = list(image)
emnist_test['image_flat'] = emnist_test['image'].apply(lambda x: np.array(x).reshape(-1))
emnist_test['label'] = label
emnist_test['class'] = class_label


# Combine the training and test splits
emnist_all = pd.concat([emnist_train, emnist_test], axis=0)
display('emnist_all')
# Subset for only digits 0-9
digits = emnist_all[emnist_all['label'] < 10]

# Subset for lowercase letters
lowercase = emnist_all[(emnist_all['class'] >= 'a') & (emnist_all['class'] <= 'z')]
uppercase = emnist_all[(emnist_all['class'] >= 'A') & (emnist_all['class'] <= 'Z')]

# Subset for upper- and lowercase letters a, b, c, d, e, f, g
a2g = emnist_all[(emnist_all['class'].isin(['a', 'b', 'c', 'd', 'e', 'f', 'g']))]

# Subset for upper- and lowercase letters a, b, c, x, y, z
abcxyz = emnist_all[(emnist_all['class'].isin(['a', 'b', 'c', 'A', 'B', 'C', \
                                               'x', 'y', 'z', 'X', 'Y', 'Z']))]

'emnist_all'

In [ ]:
# Display the size of a2g, abcxyz, digits, and the full dataset
display(Markdown(f'# Dataset Sizes'))
display(Markdown(f'**a2g**: {len(a2g)}'))
display(Markdown(f'**abcxyz**: {len(abcxyz)}'))
display(Markdown(f'**digits**: {len(digits)}'))
display(Markdown(f'**emnist_all**: {len(emnist_all)}'))

In [ ]:
# FIXME: Classify lettters as uppercase/lowercase
abcxyz['is_upper'] = abcxyz['class'].apply(lambda x: x.isupper())
abcxyz['is_lower'] = abcxyz['class'].apply(lambda x: x.islower())

# FIXME: Classify digits as even/odd
digits['is_even'] = digits['label'] % 2 == 0
digits['is_odd'] = digits['label'] % 2 != 0


In [ ]:
symbols_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g']

mask_train = emnist_train['class'].apply(lambda x: x in symbols_list)
train_01 = emnist_train[mask_train]
train_01.reset_index(drop=True, inplace=True)

mask_valid = emnist_test['class'].apply(lambda x: x in symbols_list)
valid_01 = emnist_test[mask_valid]
valid_01.reset_index(drop=True, inplace=True)

metrics_dict = {
    'a2g' : { # task name (a2g classifier)
        'logistic_regression': {
            'confusion_matrix': [],
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        },
        'xgboost': {
            'confusion_matrix': [],
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        },
        'random_forest': {
            'confusion_matrix': [],
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        },
        'neural_network': {
            'confusion_matrix': [],
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        }
    }
}

task = 'a2g'
model_name = 'random_forest'
metrics_dict[task] = {model_name: {}}

# Initialize random forest classifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train and evaluate model
rf_clf.fit(train_01['image_flat'].tolist(), train_01['label'])
y_pred = rf_clf.predict(valid_01['image_flat'].tolist())

acc = accuracy_score(valid_01['label'], y_pred)
prec = precision_score(valid_01['label'], y_pred, average='micro')
rec = recall_score(valid_01['label'], y_pred, average='micro')
f1 = f1_score(valid_01['label'], y_pred, average='micro')
cm = confusion_matrix(valid_01['label'], y_pred)

# Store performance metrics in dictionary
metrics_dict[task][model_name] = {'accuracy': acc,
                                  'precision': prec,
                                  'recall': rec,
                                  'f1': f1,
                                  'confusion_matrix': cm}

# Display performance metrics
display_metrics(task, model_name, metrics_dict)

In [ ]:

# Exclude easily confused symbols
excluded_symbols = ['O', '0']

# Filter the data to exclude easily confused symbols
valid_filtered = valid_01[~valid_01['class'].isin(excluded_symbols)]

# Predict labels for the filtered data
y_pred_filtered = rf_clf.predict(valid_filtered['image_flat'].tolist())

# Calculate performance metrics for each class
performance_metrics = {}
for symbol in symbols_list:
    if symbol not in excluded_symbols:
        symbol_indices = valid_filtered['class'] == symbol
        symbol_labels = valid_filtered.loc[symbol_indices, 'label']
        symbol_predictions = y_pred_filtered[symbol_indices]
        accuracy = accuracy_score(symbol_labels, symbol_predictions)
        precision = precision_score(symbol_labels, symbol_predictions, average='micro')
        recall = recall_score(symbol_labels, symbol_predictions, average='micro')
        f1 = f1_score(symbol_labels, symbol_predictions, average='micro')
        performance_metrics[symbol] = {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

# Sort the performance metrics by accuracy in descending order
sorted_metrics = sorted(performance_metrics.items(), key=lambda x: x[1]['accuracy'], reverse=True)

# Display the performance metrics
for symbol, metrics in sorted_metrics:
    print(f"Symbol: {symbol}")
    print(f"Accuracy: {metrics['accuracy']}")
    print(f"Precision: {metrics['precision']}")
    print(f"Recall: {metrics['recall']}")
    print(f"F1 Score: {metrics['f1']}")
    print("--------------------")

# Plot the confusion matrix
cm_filtered = confusion_matrix(valid_filtered['label'], y_pred_filtered)
plt.figure(figsize=(10, 8))
sns.heatmap(cm_filtered, annot=True, fmt='d', cmap='Blues', xticklabels=symbols_list, yticklabels=symbols_list)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Excluding Easily Confused Symbols)')
plt.show()


In [ ]:
# Perform a full model training and hyperparameter tuning.

# 1. Select candidate models, hyperparameter options, and evaluation metric
# 2. Set aside a validation hold-out dataset

# abcxyz = emnist_all[(emnist_all['class'].isin(['a', 'b', 'c', 'A', 'B', 'C', 'x', 'y', 'z', 'X', 'Y', 'Z']))]
# abcxyz Classifier: XGBoost
task = 'abcxyz'
model_name = 'xgboost'
metrics_dict2 = {
    'abcxyz' : { # task name (abcxyz classifier)
        'logistic_regression': {
            'confusion_matrix': [],
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        },
        'xgboost': {
            'confusion_matrix': [],
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        },
        'random_forest': {
            'confusion_matrix': [],
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        },
        'neural_network': {
            'confusion_matrix': [],
            'accuracy': [],
            'precision': [],
            'recall': [],
            'f1': []
        }
    }
}

symbols_list2 = ['a', 'b', 'c', 'A', 'B', 'C', 'x', 'y', 'z', 'X', 'Y', 'Z']

mask_train2 = emnist_train['class'].apply(lambda x: x in symbols_list2)
label_mapping = {10: 0, 11: 1, 12: 2, 33: 3, 34: 4, 35: 5, 36: 6, 37: 7, 38: 8, 59: 9, 60: 10, 61: 11}
emnist_train['label'] = emnist_train['label'].map(label_mapping)
train_02 = emnist_train[mask_train2]
train_02.reset_index(drop=True, inplace=True)

mask_valid2 = emnist_test['class'].apply(lambda x: x in symbols_list2)
emnist_test['label'] = emnist_test['label'].map(label_mapping)
valid_02 = emnist_test[mask_valid2]
valid_02.reset_index(drop=True, inplace=True)

# Initialize XGBoost classifier
xgb_clf = XGBClassifier(n_estimators=100, random_state=42)

# Train and evaluate model
xgb_clf.fit(train_02['image_flat'].tolist(), train_02['label'])
y_pred = xgb_clf.predict(valid_02['image_flat'].tolist())

# Calculate performance metrics
acc = accuracy_score(valid_02['label'], y_pred)
prec = precision_score(valid_02['label'], y_pred, average='weighted')
rec = recall_score(valid_02['label'], y_pred, average='weighted')
f1 = f1_score(valid_02['label'], y_pred, average='weighted')
cm = confusion_matrix(valid_02['label'], y_pred)

# Store performance metrics in dictionary
metrics_dict2[task][model_name] = {'accuracy': acc,
                                  'precision': prec,
                                  'recall': rec,
                                  'f1': f1,
                                  'confusion_matrix': cm}

# Display performance metrics
display_metrics2(task, model_name, metrics_dict2)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, KFold, ParameterGrid, train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

X=abcxyz['image_flat'].tolist()
y=abcxyz['class'].tolist()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
label_mapping2 = {10: 0, 11: 1, 12: 2, 33: 3, 34: 4, 35: 5, 36: 6, 37: 7, 38: 8, 59: 9, 60: 10, 61: 11}
emnist_train['label'] = emnist_train['label'].map(label_mapping)

best_score = 0
best_model = None
rf_param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20]}
xgb_param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.1, 0.2]}
scoring_metric = 'accuracy'
models = [
    (RandomForestClassifier(), rf_param_grid),
    (XGBClassifier(), xgb_param_grid)
]
# If you want to use cross_val_score instead of doing above manually
		# Using sklearn's function.
for model, param_grid in models:
    for param_combination in ParameterGrid(param_grid):
        model.set_params(**param_combination)
 # cross_val_score returns array of scores, which you need to take the avg
        scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
        avg_score = np.mean(scores)
        if avg_score > best_score:
            best_score = avg_score
    best_model = model 


print(f"Best Model Parameters: {best_model}")
print(f"Best Model Average Cross-Validation Score: {best_score}")

best_model.fit(X_train, y_train)

y_val_pred = best_model.predict(X_val)
validation_score = accuracy_score(y_val, y_val_pred)
print(f"Validation Hold-out Score: {validation_score}")

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/petra/Desktop/study/UCSF/datasci 223 python/datasci_223/.conda/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/petra/Desktop/study/UCSF/datasci 223 python/datasci_223/.conda/lib/python3.11/site-packages/xgboost/core.py", line 730, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "/Users/petra/Desktop/study/UCSF/datasci 223 python/datasci_223/.conda/lib/python3.11/site-packages/xgboost/sklearn.py", line 1471, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values o